<a href="https://colab.research.google.com/github/kk98033/NTTU-Digital-System-Design-Lab-Project/blob/main/%E9%80%8F%E9%81%8E_RAG_%E6%8A%80%E8%A1%93%E5%BC%B7%E5%8C%96_LLM_%E8%B3%87%E8%A8%8A%E6%AA%A2%E7%B4%A2%E4%B8%A6%E6%98%8E%E7%A2%BA%E5%91%88%E7%8F%BE%E4%BE%86%E6%BA%90%E8%BF%BD%E6%BA%AF(updated%2C%20%E6%B2%92%E6%9C%89%E7%94%A8milvus%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.9 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

# 設定OpenAI API密鑰
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import CitationQueryEngine
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext

from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter

In [4]:
!wget 'https://drive.google.com/uc?export=download&id=1kDwfFbMC3nM0K7OTXu88cczYBFm10pQT' -O 原住民資料.pdf
!wget 'https://drive.google.com/uc?export=download&id=1OIK0zUqw7iwuhtvAmLwUnz1-aNqPx7pZ' -O 原住民資料2.pdf

--2024-04-19 02:10:16--  https://drive.google.com/uc?export=download&id=1kDwfFbMC3nM0K7OTXu88cczYBFm10pQT
Resolving drive.google.com (drive.google.com)... 142.250.99.102, 142.250.99.113, 142.250.99.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.99.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1kDwfFbMC3nM0K7OTXu88cczYBFm10pQT&export=download [following]
--2024-04-19 02:10:16--  https://drive.usercontent.google.com/download?id=1kDwfFbMC3nM0K7OTXu88cczYBFm10pQT&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.99.132, 2607:f8b0:400e:c0c::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.99.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5073207 (4.8M) [application/octet-stream]
Saving to: ‘原住民資料.pdf’

原住民資料.pdf      100%[===================>]   4.84M  23.5MB/s    in 0.2

In [5]:
# 從 PDF 文件中讀取文檔數據。
documents = SimpleDirectoryReader(
    input_files=["./原住民資料.pdf", "./原住民資料2.pdf"]
  ).load_data()
print(len(documents))

618


In [6]:
# 從文檔中創建向量存儲索引。
index = VectorStoreIndex.from_documents(documents)

In [19]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# 設置 LLaMA 模型、嵌入模型、節點解析器等。
Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

In [20]:
# 初始化引擎，設置相似性搜索的參數。
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    citation_chunk_size=512,
)

# 測試效果

In [26]:
response = query_engine.query("阿美族有什麼祭典，請詳細講解")
print(response)

阿美族有豐年祭和漁撈祭兩個主要祭典。豐年祭是一項綜合活動，包括豐收、謝神、聯誼、社交、年齡階級晉級儀式、軍事訓練驗收等多方面內容，具有經濟、宗教、社會、政治、文化等多性質的意義，並且是具有多元文化特質的盛大活動[1]。漁撈祭則包含海祭和河祭兩種形式，是在每年5、6月間祭祀海神或河川神的活動，族人藉此祈求出海平安或撈捕魚類滿載而歸。漁撈祭具有敬老尊賢的意義，祭典由青年以魚蟹、米酒祭拜河神或海神後揭開序幕，並按照年齡階級的輩份順序將煮熟的漁獲送交長者、耆老品嚐，以表示老者優先，並賞賜漁獲給表現良好的年輕人，體現互助共享和重視長者等倫理觀念[2]。


In [27]:
for source in response.source_nodes:
    print(source.node.get_text())

Source 1:
年7 月開始，各部落按照稻米收成時間由南往北安排辦理，祭典時間為期1 ∼ 7 天之間。   豐年祭雖然以豐收為名，但內容包括豐收、謝神、聯誼、社交與年齡階級晉級儀式、軍事訓練驗收儀式等，是具經濟、宗教、社會、政治、文化等多性質的綜合活動。豐年祭的多面向活動，具有多元文化特質的意義，加上參與人數眾多，規模相當盛大。阿美族人即使移居都市，仍然持續辦理豐年祭典，傳承各項傳統文化與觀念，這個祭典也是新一代族人對部落文化認同的重要活動。 2. 漁撈祭 用傳統檳榔葉柄食盒烹煮魚 阿美族的漁撈祭包含海祭與河祭，是在每年5、6 月間祭祀海神或河川神的活動，族人藉此祈求出海平安或撈捕魚類滿載而歸。

Source 2:
漁撈祭典有不同的名稱，在海邊進行的海祭，北部阿美稱為mia’adis，海岸阿美族稱為misacepo’，馬蘭阿美族稱為mikesi’；撈捕淡水魚的河川祭，沿秀姑巒溪兩岸的阿美族人稱為komoris。   而都歷部落於民國70 年（1981） 中斷過海祭， 復於民國100 年（2011）時恢復此祭儀，更名為pafafuy。 傳統領袖帶領族人祭祖 漁撈祭具有敬老尊賢的意義，祭典由青年以魚蟹、米酒祭拜河神或海神後揭開序幕，接著由各年齡階層進入海河溪流中撈捕魚類。近午時分，青年將捕獲的魚類集中、烹煮，並按照年齡階級的輩份順序，將煮熟的漁獲送交長者、耆老品嚐，以表示老者優先，而年長者也會賞賜漁獲

Source 3:
祈求作物豐收，也祈使身體健康，如未婚女子在播種祭時盪鞦韆，便能在婚後早生子女，若是已婚而尚未生育者，則能迅速懷孕。因此既是祈福，又是娛樂。         第二天，由先生媽主持告祖祭儀，祭祀邵族的祖靈，祈求作物能順利成長，以及族人能平安健康。第三天，族人仍休息不工作，依舊飲酒為樂。直到第四天的清晨，族人才上山播種，當播種至一半時須由先生媽再主持一次祭祀儀式，以糯米糕作祭品，續求作物的成長及為族人祈福。至此，播種祭始告全部結束。 參考資料  原住民族委員會→原住民族簡介→邵族→祭典傳說：http://www.apc.gov.tw/portal/docList.html?CID=AC58C79198E1FD34&type=1EE2C9E1BA3440B2D0636733C6861689 阿美族總人口220,067，阿美族人群聚而居，部落規模大、人口

## 取得資訊來源詳細資訊

In [46]:
for source in response.source_nodes:
    print(source.node.get_text())
    print(source.node.get_metadata_str())
    print(source.node.node_id)
    print()

Source 1:
年7 月開始，各部落按照稻米收成時間由南往北安排辦理，祭典時間為期1 ∼ 7 天之間。   豐年祭雖然以豐收為名，但內容包括豐收、謝神、聯誼、社交與年齡階級晉級儀式、軍事訓練驗收儀式等，是具經濟、宗教、社會、政治、文化等多性質的綜合活動。豐年祭的多面向活動，具有多元文化特質的意義，加上參與人數眾多，規模相當盛大。阿美族人即使移居都市，仍然持續辦理豐年祭典，傳承各項傳統文化與觀念，這個祭典也是新一代族人對部落文化認同的重要活動。 2. 漁撈祭 用傳統檳榔葉柄食盒烹煮魚 阿美族的漁撈祭包含海祭與河祭，是在每年5、6 月間祭祀海神或河川神的活動，族人藉此祈求出海平安或撈捕魚類滿載而歸。

page_label: 71
file_name: 原住民資料.pdf
file_path: 原住民資料.pdf
file_type: application/pdf
file_size: 5073207
creation_date: 2024-04-19
last_modified_date: 2024-04-03
ef1d43c4-b74d-4e66-b884-d23e341aaf1c

Source 2:
漁撈祭典有不同的名稱，在海邊進行的海祭，北部阿美稱為mia’adis，海岸阿美族稱為misacepo’，馬蘭阿美族稱為mikesi’；撈捕淡水魚的河川祭，沿秀姑巒溪兩岸的阿美族人稱為komoris。   而都歷部落於民國70 年（1981） 中斷過海祭， 復於民國100 年（2011）時恢復此祭儀，更名為pafafuy。 傳統領袖帶領族人祭祖 漁撈祭具有敬老尊賢的意義，祭典由青年以魚蟹、米酒祭拜河神或海神後揭開序幕，接著由各年齡階層進入海河溪流中撈捕魚類。近午時分，青年將捕獲的魚類集中、烹煮，並按照年齡階級的輩份順序，將煮熟的漁獲送交長者、耆老品嚐，以表示老者優先，而年長者也會賞賜漁獲

page_label: 71
file_name: 原住民資料.pdf
file_path: 原住民資料.pdf
file_type: application/pdf
file_size: 5073207
creation_date: 2024-04-19
last_modified_date: 2024-04-03
ef1d43c4-b74d-4e66-b884-d23e3

In [47]:
import json

In [48]:
print(response.metadata)
print(json.dumps(response.metadata, indent=4, sort_keys=True, ensure_ascii=False))

{'ef1d43c4-b74d-4e66-b884-d23e341aaf1c': {'page_label': '71', 'file_name': '原住民資料.pdf', 'file_path': '原住民資料.pdf', 'file_type': 'application/pdf', 'file_size': 5073207, 'creation_date': '2024-04-19', 'last_modified_date': '2024-04-03'}, '43831d09-7987-440f-b06f-8b02b85768b3': {'page_label': '64', 'file_name': '原住民資料.pdf', 'file_path': '原住民資料.pdf', 'file_type': 'application/pdf', 'file_size': 5073207, 'creation_date': '2024-04-19', 'last_modified_date': '2024-04-03'}, 'e9321498-b697-47e7-88d8-33764283c5bb': {'page_label': '419', 'file_name': '原住民資料.pdf', 'file_path': '原住民資料.pdf', 'file_type': 'application/pdf', 'file_size': 5073207, 'creation_date': '2024-04-19', 'last_modified_date': '2024-04-03'}}
{
    "43831d09-7987-440f-b06f-8b02b85768b3": {
        "creation_date": "2024-04-19",
        "file_name": "原住民資料.pdf",
        "file_path": "原住民資料.pdf",
        "file_size": 5073207,
        "file_type": "application/pdf",
        "last_modified_date": "2024-04-03",
        "page_label": "6

In [53]:
for source in response.source_nodes:
    sourceMetadata = response.metadata[source.node.node_id]
    print(json.dumps(sourceMetadata, indent=4, sort_keys=True, ensure_ascii=False))
    print()

{
    "creation_date": "2024-04-19",
    "file_name": "原住民資料.pdf",
    "file_path": "原住民資料.pdf",
    "file_size": 5073207,
    "file_type": "application/pdf",
    "last_modified_date": "2024-04-03",
    "page_label": "71"
}

{
    "creation_date": "2024-04-19",
    "file_name": "原住民資料.pdf",
    "file_path": "原住民資料.pdf",
    "file_size": 5073207,
    "file_type": "application/pdf",
    "last_modified_date": "2024-04-03",
    "page_label": "71"
}

{
    "creation_date": "2024-04-19",
    "file_name": "原住民資料.pdf",
    "file_path": "原住民資料.pdf",
    "file_size": 5073207,
    "file_type": "application/pdf",
    "last_modified_date": "2024-04-03",
    "page_label": "64"
}

{
    "creation_date": "2024-04-19",
    "file_name": "原住民資料.pdf",
    "file_path": "原住民資料.pdf",
    "file_size": 5073207,
    "file_type": "application/pdf",
    "last_modified_date": "2024-04-03",
    "page_label": "64"
}

{
    "creation_date": "2024-04-19",
    "file_name": "原住民資料.pdf",
    "file_path": "原住民資料.pdf",
   

In [54]:
response = query_engine.query("台灣最高的樓是什麼")

In [56]:
print(response)
print('=================')
for source in response.source_nodes:
    print(source.get_text())

None of the provided sources mention the tallest building in Taiwan.
Source 1:
嶺」是本區最高的山嶺，海拔 2,400 公尺，南方有「藤包山」，海拔 2,200 公尺，是對卡族人甚具代表性的地方，已逐漸被後輩的卡族人視為「聖山」。        民國 98 年「莫拉克 88 風災」，由於水量太大，楠梓仙溪重創，下游的小林村因而滅村。卡族人也有數戶受災，被安遷至杉林區的「大愛園區」。所以現今的卡族人分別居住在那瑪夏區的達卡努瓦里、瑪雅里、大愛園區及因就業、就學散居各地的族人。 生活圖像        卡族傳統的經濟生活以農耕燒墾為主，狩獵捕魚為輔。傳統作物有小米、旱稻、糯稻、地瓜、芋頭、玉米等。狩獵上以男性擔任為主，狩獵期間約於 9月至隔年 4 月，約為農閒期間，可分個人與團體。個人的狩獵目的為家庭食物覓尋；團體則大部分為祭儀所需。另捕魚則有刺、網、釣、毒、圍渠等方式。

Source 2:
工藝        工藝的表現有木工，如製作木臼、木桶、蒸桶、木揹架、木枕、木凳、杵、杖等。藤竹編工，如置穀籮筐、水竹筒、竹背簍、藤竹

Source 3:
► 竹壁式        竹壁式家屋，以竹材為主要建材，從地平面往上構築，使用茅草覆蓋在屋頂上。  藝術 ► 木琴        木琴是太魯閣族獨特的樂器，木琴製作的材料有：食茱萸（sangas）、 山 鹽 木 （prihut）、 檜 木 （qulit、byugu、plux）、 油 桐（bruqil）、 楓 木 （dgarung）等，以山鹽木之打擊聲音最清脆，而油桐之聲音最厚實。通常在製作前需要選擇長時間陰乾的木材，才能讓木琴不容易變形。        木琴的敲擊演奏用於召喚親友共享美食或作為舞蹈伴奏，敲擊時男士盤坐、女士則採跪姿進行，可以分為單手或雙手演奏兩種。木琴的音階有Re、Mi、Sol、La 等四個音，音階的位置分別是 1 號 Re 音（圓 5.5cm）、2 號 Mi 音（6.5cm）、3 號 Sol 音（5.2cm）、4 號 La 音（4.8cm）。

Source 4:
牆壁以大量的茅管直立排列後，並以藤條綁住一層或兩層。 3.屋頂如橢圓形的傘斜搭而下，並在上面蓋上茅草，自高處俯視有如母雞孵卵時背脊的形狀。邊延伸出壁外二、三尺，形成四周